- Sky writes this in 21 Jan 2026
- The aim is to detect AprilTags in images using Python

In [4]:
from pyapriltags import Detector
import numpy
import os

test_images_path = './test_files'

visualization = True
try:
    import cv2
except:
    raise Exception('You need cv2 in order to run the demo. However, you can still use the library without it.')

try:
    from cv2 import imshow
except:
    print("The function imshow was not implemented in this installation. Rebuild OpenCV from source to use it")
    print("VIsualization will be disabled.")
    visualization = False

try:
    import yaml
except:
    raise Exception('You need yaml in order to run the tests. However, you can still use the library without it.')


In [5]:
at_detector = Detector(families='tag36h11',
                       nthreads=1, # 线程数
                       quad_decimate=1.0, #  不降采样
                       quad_sigma=0.0, # 高斯模糊的σ
                       refine_edges=1, # 边缘亚像素级优化
                       decode_sharpening=0.25, # 锐化增强
                       debug=0) # 是否输出调试图像

with open(test_images_path + '/test_info.yaml', 'r') as stream:
    # parameters = yaml.load(stream)
    parameters = yaml.safe_load(stream)

## 1 Single sample image

In [6]:
#### test WITH THE SAMPLE IMAGE ####

print("\n\nTESTING WITH A SAMPLE IMAGE")

img = cv2.imread(test_images_path+'/'+parameters['sample_test']['file'], cv2.IMREAD_GRAYSCALE)
cameraMatrix = numpy.array(parameters['sample_test']['K']).reshape((3,3))
camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )
# fx, fy, cx, cy
if visualization:
    cv2.imshow('Original image',img)



TESTING WITH A SAMPLE IMAGE


In [7]:
tags = at_detector.detect(img, True, camera_params, parameters['sample_test']['tag_size'])
print(tags)

color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

for tag in tags:
    for idx in range(len(tag.corners)):
        cv2.line(color_img, tuple(tag.corners[idx-1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))

    cv2.putText(color_img, str(tag.tag_id),
                org=(tag.corners[0, 0].astype(int)+10,tag.corners[0, 1].astype(int)+10),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.8,
                color=(0, 0, 255))

if visualization:
    cv2.imshow('Detected tags', color_img)

    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()


[Detection object:
tag_family = b'tag36h11'
tag_id = 60
tag_size = 0.065
hamming = 0
decision_margin = 24.63052749633789
homography = [[ 1.06447279e+01  1.35911965e+01  1.50569691e+02]
 [-7.48612659e-03  1.33928712e+01  9.97183221e+01]
 [ 3.33517876e-03  3.97460305e-02  1.00000000e+00]]
center = [150.56969059  99.71832209]
corners = [[148.12287903 109.14463043]
 [167.58581543 108.43231201]
 [153.20141602  89.57963562]
 [132.02140808  90.2197113 ]]
pose_R = [[ 0.97181644  0.17516465  0.1577661 ]
 [ 0.04936332  0.50319692 -0.86276075]
 [-0.23051261  0.84623295  0.48036833]]
pose_t = [[-0.49579828]
 [-0.30725303]
 [ 0.91252828]]
pose_err = 9.537329008176483e-07
, Detection object:
tag_family = b'tag36h11'
tag_id = 82
tag_size = 0.065
hamming = 0
decision_margin = 24.103368759155273
homography = [[ 8.44839326e+00  7.18513601e+00  4.66607972e+02]
 [-8.15325677e-01  1.13323854e+01  8.20929561e+01]
 [-3.09552474e-03  2.75834816e-02  1.00000000e+00]]
center = [466.60797167  82.0929561 ]
corner

## 2 旋转图片

In [8]:
#### test WITH THE ROTATION IMAGES ####

import time

print("\n\nTESTING WITH ROTATION IMAGES")

time_num = 0
time_sum = 0

# test_images_path = 'test_files'
image_names = parameters['rotation_test']['files']

for image_name in image_names:
    print("Testing image ", image_name)
    ab_path = test_images_path + '/' + image_name
    if(not os.path.isfile(ab_path)):
        continue
    groundtruth = float(image_name.split('_')[-1].split('.')[0])  # name of test_files image should be set to its groundtruth

    parameters['rotation_test']['rotz'] = groundtruth
    cameraMatrix = numpy.array(parameters['rotation_test']['K']).reshape((3,3))
    camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )

    img = cv2.imread(ab_path, cv2.IMREAD_GRAYSCALE)

    start = time.time()
    tags = at_detector.detect(img, True, camera_params, parameters['rotation_test']['tag_size'])

    time_sum+=time.time()-start
    time_num+=1
    print(time_num)
    print(tags[0].pose_t, parameters['rotation_test']['posx'], parameters['rotation_test']['posy'], parameters['rotation_test']['posz'])
    print(tags[0].pose_R, parameters['rotation_test']['rotx'], parameters['rotation_test']['roty'], parameters['rotation_test']['rotz'])

print("AVG time per detection: ", time_sum/time_num)




TESTING WITH ROTATION IMAGES
Testing image  test_image_rotation_0.png
1
[[ 0.00853323]
 [-0.03376011]
 [ 0.20404375]] 0.3 0.0 0.05
[[ 0.99953174 -0.02025914  0.02293174]
 [ 0.02532478  0.96833778 -0.2483558 ]
 [-0.0171742   0.24882025  0.9683974 ]] 0.0 0.0 0.0
Testing image  test_image_rotation_10.png
2
[[ 0.00897768]
 [-0.03391492]
 [ 0.20457868]] 0.3 0.0 0.05
[[ 0.99559287 -0.0180798  -0.09202153]
 [-0.0059657   0.96704328 -0.25454216]
 [ 0.09359087  0.25396934  0.96267353]] 0.0 0.0 10.0
Testing image  test_image_rotation_20.png
3
[[ 0.01026743]
 [-0.0343606 ]
 [ 0.20644286]] 0.3 0.0 0.05
[[ 0.96045814 -0.00979511 -0.27825208]
 [-0.05978911  0.96881051 -0.24048173]
 [ 0.27192909  0.24760908  0.92991629]] 0.0 0.0 20.0
Testing image  test_image_rotation_30.png
4
[[ 0.01173394]
 [-0.03444123]
 [ 0.20705484]] 0.3 0.0 0.05
[[ 0.90032139 -0.01421019 -0.43499364]
 [-0.09591059  0.96841844 -0.23014535]
 [ 0.42452627  0.24892528  0.87052493]] 0.0 0.0 30.0
Testing image  test_image_rotation_

In [ ]:
#### test WITH MULTIPLE TAGS IMAGES ####

print("\n\nTESTING WITH MULTIPLE TAGS IMAGES")

time_num = 0
time_sum = 0

image_names = parameters['multiple_tags_test']['files']

for image_name in image_names:
    print("Testing image ", image_name)
    ab_path = test_images_path + '/' + image_name
    if(not os.path.isfile(ab_path)):
        continue

    cameraMatrix = numpy.array(parameters['multiple_tags_test']['K']).reshape((3,3))
    camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )

    img = cv2.imread(ab_path, cv2.IMREAD_GRAYSCALE)

    start = time.time()
    tags = at_detector.detect(img, True, camera_params, parameters['multiple_tags_test']['tag_size'])
    time_sum+=time.time()-start
    time_num+=1

    tag_ids = [tag.tag_id for tag in tags]
    print(len(tags), " tags found: ", tag_ids)


    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    for tag in tags:
        for idx in range(len(tag.corners)):
            cv2.line(color_img, tuple(tag.corners[idx-1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))

        cv2.putText(color_img, str(tag.tag_id),
                    org=(tag.corners[0, 0].astype(int)+10,tag.corners[0, 1].astype(int)+10),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.8,
                    color=(0, 0, 255))

    if visualization:
        cv2.imshow('Detected tags for ' + image_name    , color_img)

        k = cv2.waitKey(0)
        if k == 27:         # wait for ESC key to exit
            cv2.destroyAllWindows()

print("AVG time per detection: ", time_sum/time_num)




TESTING WITH MULTIPLE TAGS IMAGES
Testing image  test_image_multiple_01.png
6  tags found:  [22, 24, 58, 85, 144, 198]
Testing image  test_image_multiple_02.png
5  tags found:  [22, 24, 85, 144, 198]
Testing image  test_image_multiple_03.png
6  tags found:  [22, 24, 58, 85, 144, 198]
Testing image  test_image_multiple_04.png
6  tags found:  [22, 24, 58, 85, 144, 198]
Testing image  test_image_multiple_05.png
4  tags found:  [22, 24, 85, 198]
Testing image  test_image_multiple_06.png
6  tags found:  [22, 24, 58, 85, 144, 198]
Testing image  test_image_multiple_07.png
6  tags found:  [22, 24, 58, 85, 144, 198]
Testing image  test_image_multiple_08.png
3  tags found:  [85, 144, 198]
Testing image  test_image_multiple_09.png
5  tags found:  [22, 24, 58, 85, 144]
Testing image  test_image_multiple_10.png
4  tags found:  [24, 58, 85, 198]
AVG time per detection:  0.024330449104309083


: 